# Vector Search Evaluator

This notebook evaluates TIG's vector search algorithms against standard academic benchmark instances.

**Useful Links:**
  * [Challenge Description](https://tig.foundation/challenges/vector_search)
  * [Challenge Code](https://github.com/tig-foundation/tig-monorepo/blob/main/tig-challenges/src/vector_search.rs)

## 1. Setup Environment

### 1.1. Install Cargo

In [ ]:
import shutil
import os
if shutil.which("cargo") is None:
    print("cargo not found. Installing Rust and Cargo...")
    !curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
    os.environ["PATH"] += f":{os.environ['HOME']}/.cargo/bin"
else:
    print("cargo is already installed.")

### 1.2. Clone tig-SOTA-metrics

In [ ]:
cwd = os.getcwd()
if cwd.endswith("tig-SOTA-metrics"):
    os.chdir("vector_search_evaluator")
elif not cwd.endswith("vector_search_evaluator"):
    !git clone https://github.com/tig-foundation/tig-SOTA-metrics
    os.chdir("tig-SOTA-metrics/vector_search_evaluator")
else:
    print("already in the vector_search_evaluator directory.")

### 1.3. Install Python Dependencies

In [ ]:
!pip install -r requirements.txt

### 1.4. Download Datasets

Currently, this evaluator has two placeholder datasets (SOTA datasets TBC):
- **SIFT** is a dataset of Scale-Invariant Feature Transform descriptors. Original source: [INRIA TEXMEX Corpus](ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz):
    - Dimensions: 128
    - Base vectors: 1,000,000
    - Query vectors: 10,000

- **Fashion-MNIST** is dataset of clothing item images encoded as vectors. Original source: [HuggingFace Dataset](https://huggingface.co/datasets/open-vdb/fashion-mnist-784-euclidean):
    - Dimensions: 784
    - Base vectors: 60,000
    - Query vectors: 10,000

In [ ]:
datasets = ["SIFT", "Fashion_MNIST"]
for ds in datasets:
    if not os.path.exists(f"data/{ds}"):
        !cd data && python3 download_{ds}.py
    else:
        print(f"{ds} dataset already downloaded.")

## 2. Perform Evaluation

### 2.1. Fetch Top Earning Algorithms

Vector Search was updated to a GPU challenge in round 74

In [ ]:
import requests

API_URL = "https://mainnet-api.tig.foundation"

print("Fetching block")
block = requests.get(f"{API_URL}/get-block").json()["block"]
curr_round = block["details"]["round"]
print(f"Current Round: {curr_round}")

algorithms = {
    x['id']: x
    for x in requests.get(f"{API_URL}/get-algorithms?block_id={block['id']}").json()["algorithms"]
}

print(f"Fetching Top Earning Algorithms for Vector Search from Rounds 74 to {curr_round - 1}")
top_algos = []
for r in range(74, curr_round):
    data = requests.get(f"{API_URL}/get-round-emissions?round={r}").json()
    a_id, earnings = max(
        filter(
            lambda x: x[0].startswith("c004"),
            map(
                lambda x: (x[0], int(x[1]) / 1e18),
                data["algorithms"].items()
            )
        ),
        key=lambda x: x[1]
    )
    a_name = algorithms[a_id]['details']['name']
    print(f"Round: {r}, Algo: {a_name}, Round Earnings: {earnings:.2f} TIG")
    top_algos.append((r, a_name))

### 2.2. (Optional) Evaluate Local Algorithm

If you want to evaluate an algorithm that has not been submitted to TIG (e.g. you are preparing Advance Submission):

1. Add your algorithm code to `src/{ALGORITHM_NAME}.rs` and `src/{ALGORITHM_NAME}.cu`
2. Uncomment, edit, and run the below cell

In [ ]:
# top_algos.append((curr_round + 4, {ALGORITHM_NAME}))

### 2.3. Run Evaluations

Evaluation results are saved to `evaluations` folder as csv files

In [ ]:
unique_algos = set(x[1] for x in top_algos)
for a in unique_algos:
  for ds in datasets:
      !bash run.sh data/{dataset} {a}

## 3. Plotting Results

Todo